In [20]:
!pip install textblob
!pip install tweepy
!pip install google.cloud

In [10]:
import tweepy
import re
import os
import pandas as pd
from google.cloud import language
from google.cloud import language_v1
from nltk.tokenize import WordPunctTokenizer
from textblob import TextBlob
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\zjw64\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
# Authentication
apiKey = "mi3IsHw1lzA9KlstGoEi4sQSu"
apiSecret = "0obn2WECJqrZt5eoYJoOY5HbHCQfa5Xe6PFW58EG3AEqF4dxUM"
accessToken = "1441571989357957122-eksbtpbuyFyQiMnIm9t3DOkILvzXLV" 
accessTokenSecret = "aLssavUn0tPln5czeoWaVcf0UYSRYAoenbyBIdf7TPvBE"
auth = tweepy.OAuthHandler(apiKey, apiSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [12]:
# Instantiates a client
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/path/to/file.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/zjw64/Desktop/ec601/project2/service-account-file.json"
client = language_v1.LanguageServiceClient()

In [15]:
#search tweets
keyword = input("Please enter keyword to search: ")
numOfTweet = int(input ("Please enter the number of tweets to analyze: "))

tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(numOfTweet)
tweet_list = []
for tweet in tweets:
    tweet_list.append(tweet.text)

tweet_list = pd.DataFrame(tweet_list)
print("search number: ",len(tweet_list))

Please enter keyword to search: red sox
Please enter the number of tweets to analyze: 500
search number:  500


In [16]:
# delete duplicates
tweet_list.drop_duplicates(inplace = True)
print("total number: ",len(tweet_list))

#Extracting text values
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]
print(tw_list.head(10))

total number:  157
                                                    0  \
0   @AlexandraLeslie @wpri12 Very sad day in Red S...   
1   RT @NHLBruins: The Boston Bruins mourn the los...   
2   RT @Eck43: Red Sox nation has lost a beloved i...   
3   RT @19fredlynn: I lost a great teammate and fr...   
4   RT @BGlobeSports: .@GlobeChadFinn: There were ...   
5   RT @joonlee: RIP Jerry Remy, forever a Red Sox...   
7   Jerry Remy, a Boston Red Sox second baseman wh...   
8   RT @RedSox: This weekend we lost our beloved J...   
10  RT @RedSox: A truly great player, broadcaster,...   
11  RT @BGlobeSports: Jerry Remy, an undersized ev...   

                                                 text  
0   @AlexandraLeslie @wpri12 Very sad day in Red S...  
1   RT @NHLBruins: The Boston Bruins mourn the los...  
2   RT @Eck43: Red Sox nation has lost a beloved i...  
3   RT @19fredlynn: I lost a great teammate and fr...  
4   RT @BGlobeSports: .@GlobeChadFinn: There were ...  
5   RT @joonlee: 

In [17]:
#Cleaning Text (RT, Punctuation etc)

#Creating new dataframe and new features
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]

#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
tw_list["text"] = tw_list.text.str.lower()
print(tw_list.head(10))

                                                    0  \
0   @AlexandraLeslie @wpri12 Very sad day in Red S...   
1   RT @NHLBruins: The Boston Bruins mourn the los...   
2   RT @Eck43: Red Sox nation has lost a beloved i...   
3   RT @19fredlynn: I lost a great teammate and fr...   
4   RT @BGlobeSports: .@GlobeChadFinn: There were ...   
5   RT @joonlee: RIP Jerry Remy, forever a Red Sox...   
7   Jerry Remy, a Boston Red Sox second baseman wh...   
8   RT @RedSox: This weekend we lost our beloved J...   
10  RT @RedSox: A truly great player, broadcaster,...   
11  RT @BGlobeSports: Jerry Remy, an undersized ev...   

                                                 text  
0                  very sad day in red sox nation      
1    the boston bruins mourn the loss of jerry rem...  
2    red sox nation has lost a beloved icon   i lo...  
3    i lost a great teammate and friend today  a t...  
4        there were so many reasons for red sox fa...  
5    rip jerry remy  forever a red s

In [18]:
# Google NLP API sentiment analyze
tw_goo = tw_list.copy(deep=True)
for index, tweet in tw_goo['text'].iteritems():
    document = language_v1.Document(content=tweet.encode('utf-8'), type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment
    tw_goo.loc[index, 'score'] = sentiment.score

    if sentiment.score >= -1.0 and sentiment.score < -0.25:
        tw_goo.loc[index, 'sentiment'] = "negative"
    elif sentiment.score >= -0.25 and sentiment.score <= 0.25:
        tw_goo.loc[index, 'sentiment'] = "neutral"
    else:
        tw_goo.loc[index, 'sentiment'] = "positive"

print(tw_goo.head(10))


#Creating new data frames for all sentiments (positive, negative and neutral)

tw_goo_negative = tw_goo[tw_goo["sentiment"]=="negative"]
tw_goo_positive = tw_goo[tw_goo["sentiment"]=="positive"]
tw_goo_neutral = tw_goo[tw_goo["sentiment"]=="neutral"]

                                                    0  \
0   @AlexandraLeslie @wpri12 Very sad day in Red S...   
1   RT @NHLBruins: The Boston Bruins mourn the los...   
2   RT @Eck43: Red Sox nation has lost a beloved i...   
3   RT @19fredlynn: I lost a great teammate and fr...   
4   RT @BGlobeSports: .@GlobeChadFinn: There were ...   
5   RT @joonlee: RIP Jerry Remy, forever a Red Sox...   
7   Jerry Remy, a Boston Red Sox second baseman wh...   
8   RT @RedSox: This weekend we lost our beloved J...   
10  RT @RedSox: A truly great player, broadcaster,...   
11  RT @BGlobeSports: Jerry Remy, an undersized ev...   

                                                 text  score sentiment  
0                  very sad day in red sox nation       -0.7  negative  
1    the boston bruins mourn the loss of jerry rem...    0.0   neutral  
2    red sox nation has lost a beloved icon   i lo...   -0.1   neutral  
3    i lost a great teammate and friend today  a t...   -0.6  negative  
4      

In [19]:
#Function for count_values_in single columns

def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

#Count_values for sentiment
print("Google NLP sentiment analyze: ")
print(count_values_in_column(tw_goo,"sentiment"))

Google NLP sentiment analyze: 
          Total  Percentage
neutral      61       38.85
negative     51       32.48
positive     45       28.66
